In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from ast import literal_eval
import numpy as np

In [2]:
df = pd.read_csv('./data/movies_metadata.csv', low_memory=True)
df.head(3)

C:\Users\oscar\AppData\Local\Temp\ipykernel_9220\110516680.py:1: DtypeWarning: Columns (0: popularity) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/movies_metadata.csv', low_memory=True)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [3]:
df= df.sample(n=len(df)//2, random_state=42)
#tuve que dividir el dataset en dos partes porque el proceso de vectorización y cálculo de similitud era muy pesado para la máquina, así que con la mitad del dataset se puede trabajar sin problemas.

In [4]:
df.info()

<class 'pandas.DataFrame'>
Index: 22733 entries, 43526 to 2598
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  22733 non-null  str    
 1   belongs_to_collection  2270 non-null   str    
 2   budget                 22733 non-null  str    
 3   genres                 22733 non-null  str    
 4   homepage               3903 non-null   str    
 5   id                     22733 non-null  str    
 6   imdb_id                22724 non-null  str    
 7   original_language      22729 non-null  str    
 8   original_title         22733 non-null  str    
 9   overview               22272 non-null  str    
 10  popularity             22730 non-null  object 
 11  poster_path            22530 non-null  str    
 12  production_companies   22732 non-null  str    
 13  production_countries   22732 non-null  str    
 14  release_date           22689 non-null  str    
 15  revenue        

In [5]:
C = df['vote_average'].mean()
m = df['vote_count'].quantile(0.90)
print(m)

161.0


In [6]:
q_movies = df.copy().loc[df['vote_count'] >= m]

In [7]:
# media ponderada
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']

    return (v/(v+m) * R) + (m/(m+v) * C)

In [8]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)
q_movies = q_movies.sort_values('score', ascending=False)

q_movies[['title', 'vote_count', 'vote_average', 'score']].head(15)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445278
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.415359
12481,The Dark Knight,12269.0,8.3,8.265087
292,Pulp Fiction,8670.0,8.3,8.250858
2211,Life Is Beautiful,3643.0,8.3,8.185916
1152,One Flew Over the Cuckoo's Nest,3001.0,8.3,8.162753
351,Forrest Gump,8147.0,8.2,8.149702
18465,The Intouchables,5410.0,8.2,8.124991
289,Leon: The Professional,4293.0,8.2,8.106180
3030,The Green Mile,4166.0,8.2,8.103426


In [9]:
df['overview'].head()

43526    A disgraced former cop, fresh off a six-year p...
6383     A young wife and mother, bored with day-to-day...
3154     While investigating a young nun's rape, a corr...
10146    A naïve young boy unknowingly becomes the pawn...
9531     The adventures of an upper-class suburban fami...
Name: overview, dtype: str

In [10]:

#Definir el objeto de la clase TF-IDF Vectorizer. Quitamos stop words de inglés
tfidf = TfidfVectorizer(stop_words='english')

#Reemplazar NaN por string vacío
df['overview'] = df['overview'].fillna('')

# Consruir la matriz TF-IDF haciendo ajustes y transformaciones
tfidf_matrix = tfidf.fit_transform(df['overview'])

#Mostrar shape
print(tfidf_matrix.shape)

(22733, 54000)


In [11]:
tfidf.get_feature_names_out()[5000:5010]

array(['besieger', 'besieges', 'besier', 'beskedne', 'beslag', 'besluit',
       'besmirched', 'besotted', 'bespectacled', 'bess'], dtype=object)

In [12]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print(cosine_sim.shape)

(22733, 22733)


In [13]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(df.index, index=df['title']).drop_duplicates()
indices[:10]

title
Small Crimes               43526
Up the Sandbox              6383
Bad Lieutenant              3154
Satan's Little Helper      10146
Sitcom                      9531
Nightlight                 33663
Guernsey                   36733
The Bells                   3396
Due cuori, una cappella    30019
Freeheld                   33321
dtype: int64

In [14]:
# Función que toma el título de una película como entrada y devuelve las películas más similares
def get_recommendations(title, cosine_sim=cosine_sim):
    # Obtener el índice de la película que coincide con el título
    idx = indices[title]

    # Obtener las puntuaciones de similitud por pares de todas las películas con esa película
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar las películas según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener las puntuaciones de las 10 películas más similares
    sim_scores = sim_scores[1:11]

    # Obtener los índices de las películas
    movie_indices = [i[0] for i in sim_scores]

    # Devolver las 10 películas más similares
    return df['title'].iloc[movie_indices]

get_recommendations("The Bells")

23268                     Broadway Rhythm
28042                 $1,000 on the Black
34991                       Johnny Hamlet
4156                   Frankie and Johnny
18609    Ghost Rider: Spirit of Vengeance
30451     Johnny Kapahala - Back on Board
22319                      Odds and Evens
3102                           Night Tide
43509                  The Haunted Castle
4157                   Frankie and Johnny
Name: title, dtype: str